### Functions

In [13]:
from bs4 import BeautifulSoup
import requests
from HTMLParser import HTMLParser
from time import sleep
import random
import csv
import hashlib
import requests
import re
from datetime import datetime

import numpy as np
import pandas as pd

# In[50]:

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()


# In[58]:
from selenium import webdriver
import urllib2
import MySQLdb

## get all the URLs for scraping!
## url composition: "http://www.nba.com/games/" + date of game from wiki + "//" + team A AT team B  + "//" + "/gameinfo.html"


In [7]:
#get url 
wiki_url = "https://en.wikipedia.org/wiki/2015–16_Golden_State_Warriors_season"
url = wiki_url

#use beautiful soup to parse more conveniently
r1 = urllib2.urlopen(url)
soup = BeautifulSoup(r1, 'html.parser')

wiki_table = soup.findAll('table', {'class':'toccolours'})

#get the relevant tables; i.e. the regular season sched and playoffs sched. We need these tables to generate the links
reg_season_sched = wiki_table[1]
playoffs_sched = wiki_table[2]



In [8]:
#regular season table
#get the date and the team acronyms for visiting the NBA play-by-play score
games_log = str(reg_season_sched).split('<tr style="background')
games_log = games_log[1:]

dates_teams = []
dates_teams_2 = []
for i in games_log:
    temp = i.split("https://watch.nba.com/game/")
    date_player = temp[1].split('"')
    dates_teams.append(date_player[0])


In [9]:
#playoffs season table

#comment this entire cell if you don't want playoff values
#get the date and the team acronyms for visiting the NBA play-by-play score
games_log = str(playoffs_sched).split('<tr style="background')

games_log = games_log[1:]

for i in games_log:
    temp = i.split("https://watch.nba.com/game/")
    date_player = temp[1].split('"')
    dates_teams_2.append(date_player[0])


In [10]:
#concatenate links from both regular season and playoffs
dates_teams = dates_teams + dates_teams_2
print(len(dates_teams))

106


### Code below gets the data of the NBA player, his team and his position. 

### End output is the NBA_teamnamepos_data.csv file

In [12]:
#get all the teams, put it in teams variable

teams = ['GSW']
for item in dates_teams:
    opp = item.split("/")[1][3:]
    
    
    if opp not in teams:
        teams.append(opp)

print(teams)
print(len(teams))


['GSW', 'HOU', 'NOP', 'SAC', 'MEM', 'MIN', 'LAC', 'DEN', 'PHX', 'UTA', 'CHA', 'TOR', 'BKN', 'IND', 'BOS', 'MIL', 'DAL', 'LAL', 'POR', 'DET', 'CLE', 'CHI', 'PHI', 'NYK', 'WAS', 'ATL', 'MIA', 'ORL', 'OKC', 'SAS']
30


In [15]:
# once you get the teams, visit each team at: https://www.basketball-reference.com/teams/<team>/2016.html
# note the [PHO, BRK CHO] is there because the enconding is different in the basketball-reference website
nba_dicts = {}
print "start date:", str(datetime.now())
for team in teams:
    
    try:
        # make sure the chromedriver redirected at the proper place
        browser = webdriver.Chrome(executable_path="C:\\Users\\jacob\\Downloads\\[Project] NBA Players Regression GSW 15-16\\chromedriver.exe") #replace with .Firefox(), or with the browser of your choice

        url2 = "https://www.basketball-reference.com/teams/" + team + "/2016.html"

        browser.get(url2) #navigate to the page
        browser.find_element_by_xpath('//*[@id="roster"]') #returns the inner HTML as a string
        roster_text = browser.execute_script("return document.body.innerHTML")
        browser.close()

        text = roster_text.split("<h2>Roster</h2>")

        rlvt_text = text[1].split("</tbody>")
        rlvt_text2 = rlvt_text[0].split("<tbody>")[1]
        rlvt_text3 = rlvt_text2.split(r'<tr data-row="')[1:]

        name_list = []
        pos_list = []
        fn_list = []
        ln_list = []


        for i in range(0,len(rlvt_text3)):
            sample = strip_tags(rlvt_text3[i])
            name_pos = re.findall(r'(\d{1,2})([\Da-z]+\s[\Da-z]+)([A-Z]{0,2})(\d-\d+)([A-Z][a-z]+\s\d{1,2},\s\d{4})', sample)

            tempvar = re.split(r'([A-Z]+)', name_pos[0][1])
            name = "".join(tempvar[:len(tempvar)-2])
            pos = tempvar[-2]
            
            templist = name.strip(" ").split(" ")
            first_name = templist.pop(0)
            last_name = " ".join(templist)
            
            #print first_name, last_name

            name_list.append(name.strip(" "))
            pos_list.append(pos.strip(" "))
            fn_list.append(first_name)
            ln_list.append(last_name)


            #print name, pos, team
            
        name_series = pd.Series(name_list)
        pos_series = pd.Series(pos_list)
        fn_series = pd.Series(fn_list)
        ln_series = pd.Series(ln_list)
        
        team_series = pd.Series([team for asd in range(0, len(name_series))])

        sample_dict = {'names': name_series, 'first_name':fn_series, 'last_name': ln_series, 'pos': pos_series, 'team':team_series}

        team_dict = pd.DataFrame(sample_dict)

        nba_dicts[team] = team_dict
        print team 
    except Exception as e:
        print "Error on ", url2
print "end date:", str(datetime.now())

 start date: 2018-03-08 14:13:14.094000
GSW
HOU
NOP
SAC
MEM
MIN
LAC
Error on  https://www.basketball-reference.com/teams/DEN/2016.html
Error on  https://www.basketball-reference.com/teams/PHX/2016.html
UTA
Error on  https://www.basketball-reference.com/teams/CHA/2016.html
TOR
Error on  https://www.basketball-reference.com/teams/BKN/2016.html
IND
BOS
MIL
DAL
LAL
POR
DET
CLE
CHI
PHI
NYK
WAS
ATL
MIA
ORL
OKC
SAS
end date: 2018-03-08 14:22:27.575000


In [16]:
#oops, turns out we still need to parse Phoenix, Brooklyn and Charlotte 
#since in basketball-references.com they are encoded differently
print "Start date:", str(datetime.now())
for team in ["DEN","PHO", "BRK", "CHO"]:
    
    try:
        browser = webdriver.Chrome(executable_path="C:\\Users\\jacob\\Downloads\\[Project] NBA Players Regression GSW 15-16\\chromedriver.exe") #replace with .Firefox(), or with the browser of your choice

        url2 = "https://www.basketball-reference.com/teams/" + team + "/2016.html"

        browser.get(url2) #navigate to the page
        browser.find_element_by_xpath('//*[@id="roster"]') #returns the inner HTML as a string
        roster_text = browser.execute_script("return document.body.innerHTML")
        browser.close()

        text = roster_text.split("<h2>Roster</h2>")

        rlvt_text = text[1].split("</tbody>")
        rlvt_text2 = rlvt_text[0].split("<tbody>")[1]
        rlvt_text3 = rlvt_text2.split(r'<tr data-row="')[1:]

        name_list = []
        pos_list = []
        fn_list = []
        ln_list = []


        for i in range(0,len(rlvt_text3)):
            sample = strip_tags(rlvt_text3[i])
            name_pos = re.findall(r'(\d{1,2})([\Da-z]+\s[\Da-z]+)([A-Z]{0,2})(\d-\d+)([A-Z][a-z]+\s\d{1,2},\s\d{4})', sample)

            tempvar = re.split(r'([A-Z]+)', name_pos[0][1])
            name = "".join(tempvar[:len(tempvar)-2])
            pos = tempvar[-2]
            
            templist = name.strip(" ").split(" ")
            first_name = templist.pop(0)
            last_name = " ".join(templist)

            name_list.append(name.strip(" "))
            pos_list.append(pos.strip(" "))
            fn_list.append(first_name)
            ln_list.append(last_name)



        name_series = pd.Series(name_list)
        pos_series = pd.Series(pos_list)
        fn_series = pd.Series(fn_list)
        ln_series = pd.Series(ln_list)
        
        team_series = pd.Series([team for asd in range(0, len(name_series))])

        sample_dict = {'names': name_series, 'first_name':fn_series, 'last_name': ln_series, 'pos': pos_series, 'team':team_series}

        team_dict = pd.DataFrame(sample_dict)

        nba_dicts[team] = team_dict
        print(team)
    except Exception as e:
        print "error:", e
print "end date:", str(datetime.now())

Start date: 2018-03-08 14:23:08.600000
DEN
PHO
BRK
CHO
end date: 2018-03-08 14:24:22.439000


In [17]:
# verify that all 30 teams are in dict
print(nba_dicts.keys(), len(nba_dicts.keys()))

#concatenate all values into one big data frame
NBA_TNAMEPOS_DF = pd.concat(nba_dicts.values(), ignore_index=True)

(['MIL', 'GSW', 'MIN', 'TOR', 'ATL', 'BOS', 'DET', 'NYK', 'DEN', 'SAC', 'POR', 'OKC', 'MIA', 'CLE', 'SAS', 'UTA', 'CHI', 'CHO', 'HOU', 'WAS', 'LAL', 'PHI', 'PHO', 'MEM', 'LAC', 'DAL', 'ORL', 'BRK', 'IND', 'NOP'], 30)


In [18]:
#save to csv
NBA_TNAMEPOS_DF.to_csv("NBA_teamnamepos_data.csv", sep=',',index=False)


In [19]:
NBA_dataset = pd.read_csv("NBA_teamnamepos_data.csv", sep=",")
NBA_dataset.head()

,first_name,last_name,names,pos,team
0,Giannis,Antetokounmpo,Giannis Antetokounmpo,PG,MIL
1,Jerryd,Bayless,Jerryd Bayless,SG,MIL
2,Michael,Carter-Williams,Michael Carter-Williams,PG,MIL
3,Chris,Copeland,Chris Copeland,PF,MIL
4,Jared,Cunningham,Jared Cunningham,SG,MIL
